In [3]:
import numpy as np
import random
import csv

In [4]:
items = []

with open('data/main_data.csv') as csvfile:    
	csvReader = csv.reader(csvfile)    
	for row in csvReader:        
		items.append(row[1])

temp = items[673:]

data = list(map(int, temp))

print(data)

[1745, 2116, 2604, 3205, 2925, 2927, 4878, 7010, 8077, 9905, 11588, 12422, 10185, 16021, 17895, 27197, 32211, 33729, 36057, 26121, 37492, 46843, 40618, 40489, 55209, 44526, 36501, 57049, 64718, 63956, 59635, 59384, 48484, 34418, 57491, 61488, 57426, 49447, 46643, 34976, 25054, 24728, 40920, 37259, 26347, 30156, 24867, 21380, 30148, 26336]


In [5]:
def generasi_awal(pop_size, gene_per_pop):
    population_size = (pop_size, gene_per_pop)
    gene = np.random.uniform(low=0.1, high=0.9, size=population_size)
    return gene

In [6]:
def day_predict(pop_size, gene_per_pop):
    day = []
    for i in range(2,pop_size):
        row = []
        for j in range(0, gene_per_pop):
            if j == 0:
                row.append(data[i-1])
            elif j == 1:
                row.append(data[i-2])
            elif j == 2:
                row.append(data[i])
        day.append(row)
    return day


In [7]:
def y_pred(pop_size, gene, day_predict, gene_per_pop):
    y_pred_indv = []
    for i in range (0,pop_size-2):
        row = []
        for j in gene:
            total = 0
            z = 0
            for k in j:
                if k<gene_per_pop-1:
                    g = k*day_predict[i][z]
                else:
                    g=k
                z=z+1
                total = total+g
            row.append(total)
        y_pred_indv.append(row)
    return y_pred_indv

In [8]:
def fitness_func(ps, day_predict, hasil_prediksi, gene_per_pop, dp):
    mape = []
    fitn = []
    data_y = dp[:, (gene_per_pop-1)]
    for i in range(0, ps):
        row = []
        for j in range(0,len(data_y)):
            if hasil_prediksi[j][i]>0:
                row.append(abs(hasil_prediksi[j][i]-data_y[j]/hasil_prediksi[j][i]))
            else:
                row.append(abs(0))
        mape.append(row)
    for i in range(0, len(mape)):
        f = 0
        for j in range(0,ps-2):
            f = f + mape[i][j]
        f=f/len(mape)
        fitn.append(1/(f+0.01))
    return fitn

In [9]:
def probability(fitness):
    prob = []
    sumF = 0
    for i in range(0,len(fitness)):
        sumF = sumF + fitness[i]
    for i in range(0,len(fitness)):
        prob.append(fitness[i]/sumF*100)
    return prob

In [10]:
def rangePc(probs):
    ranges = []
    x = 0
    y = 0
    minArray = []
    maxArray = []
    for i in range(0, len(probs)):
        if i == 0:
            x = 0
        else:
            x = x + probs[i-1]
        y = y + probs[i]
        minArray.append(x)
        maxArray.append(y)
    ranges = np.vstack((minArray, maxArray)).T
    return ranges

In [11]:
def round_wheel(pc, fitness, ranges):
    rW = []
    for i in range(0, int(pc * 50)):
        rNum = random.uniform(0, 100)
        for j in range(0, len(ranges)):
            if rNum >= ranges[j][0] and rNum <= ranges[j][1]:
                rW.append(j)
    return rW

In [12]:
def cross_over(pc, rw, gA):
    co = []
    x = 0
    y = 0
    for i in range(0, len(rw)):
        row = []
        for j in range(0, gene_per_pop):
            row.append(gA[rw[i]][j])
        co.append(row)
    for i in range(0, len(rw),2):
        if i % 2 == 0 and i == len(rw)-1:
            co[i][1], co[0][1] = (co[0][1] * (1-pc)) + (co[i][1] * pc), (co[i][1] * (1-pc)) + (co[0][1] * pc)
        else:
            co[i][1], co[i + 1][1] = (co[i + 1][1] * (1-pc)) + (co[i][1] * pc), (co[i][1] * (1-pc)) + (co[i + 1][1] * pc)
    return co

In [13]:
def evalGen(generasiAwal, gene_per_pop, fitness, offSpring, fitnessOffspring):
    evalIndv = []
    indvAwal = []
    indvOffspring = []
    for i in range(0, len(generasiAwal)):
        row = []
        for j in range(0, gene_per_pop+1):
            if j == gene_per_pop:
                row.append(fitness[i])
            else:
                row.append(generasiAwal[i][j])
        indvAwal.append(row)
    for i in range(0, len(offSpring)):
        row = []
        for j in range(0, gene_per_pop+1):
            if j == gene_per_pop:
                row.append(fitnessOffspring[i])
            else:
                row.append(offSpring[i][j])
        indvOffspring.append(row)
    evalIndv = indvAwal + indvOffspring
    return sorted(evalIndv, key=lambda x: x[3], reverse=True)

In [14]:
def eval_elitism(evalIndividu, pop_size):
    for i in range(0, len(evalIndividu)-pop_size):
        evalIndividu.pop()
    return evalIndividu

In [15]:
pop_size = 50
gene_per_pop = 3
pc = 0.7
generasiAwal = generasi_awal(pop_size, gene_per_pop)
generasiAwal

array([[0.63439826, 0.22360901, 0.15119697],
       [0.27686674, 0.53543589, 0.83118009],
       [0.56375247, 0.80090708, 0.51052964],
       [0.80295585, 0.78124573, 0.71998829],
       [0.37553855, 0.38232428, 0.41044578],
       [0.54118233, 0.89020048, 0.3555493 ],
       [0.72051849, 0.34532177, 0.88666676],
       [0.53061955, 0.76603652, 0.55337988],
       [0.68311977, 0.20167682, 0.30054188],
       [0.83364775, 0.22351743, 0.76144481],
       [0.19565555, 0.7355652 , 0.70936654],
       [0.54591291, 0.27868682, 0.45986345],
       [0.61841822, 0.84113295, 0.69269213],
       [0.68806064, 0.34511232, 0.42224979],
       [0.79355848, 0.39557097, 0.54206117],
       [0.31677733, 0.2706043 , 0.34508369],
       [0.32159332, 0.72347979, 0.68147735],
       [0.41852364, 0.19056163, 0.80083321],
       [0.65495062, 0.60999021, 0.30824596],
       [0.56869002, 0.8570702 , 0.37154631],
       [0.69259976, 0.40600126, 0.79101469],
       [0.68804056, 0.15559238, 0.4401014 ],
       [0.

In [16]:
dayPredict = day_predict(pop_size, gene_per_pop)
dayPredict = [list( map(int, i) ) for i in dayPredict]
print (dayPredict)

[[2116, 1745, 2604], [2604, 2116, 3205], [3205, 2604, 2925], [2925, 3205, 2927], [2927, 2925, 4878], [4878, 2927, 7010], [7010, 4878, 8077], [8077, 7010, 9905], [9905, 8077, 11588], [11588, 9905, 12422], [12422, 11588, 10185], [10185, 12422, 16021], [16021, 10185, 17895], [17895, 16021, 27197], [27197, 17895, 32211], [32211, 27197, 33729], [33729, 32211, 36057], [36057, 33729, 26121], [26121, 36057, 37492], [37492, 26121, 46843], [46843, 37492, 40618], [40618, 46843, 40489], [40489, 40618, 55209], [55209, 40489, 44526], [44526, 55209, 36501], [36501, 44526, 57049], [57049, 36501, 64718], [64718, 57049, 63956], [63956, 64718, 59635], [59635, 63956, 59384], [59384, 59635, 48484], [48484, 59384, 34418], [34418, 48484, 57491], [57491, 34418, 61488], [61488, 57491, 57426], [57426, 61488, 49447], [49447, 57426, 46643], [46643, 49447, 34976], [34976, 46643, 25054], [25054, 34976, 24728], [24728, 25054, 40920], [40920, 24728, 37259], [37259, 40920, 26347], [26347, 37259, 30156], [30156, 26347,

In [17]:
yPred = y_pred(pop_size, generasiAwal, dayPredict, gene_per_pop)
yPred

[[2126.3013455026503,
  3684.578618738719,
  3919.9022820338027,
  4937.177890449158,
  2530.5962405004934,
  3624.3920291113777,
  4436.0838628483525,
  3900.5259014001217,
  2580.0185406377905,
  4136.838834033049,
  3544.758905930239,
  2838.9446541280713,
  4580.120270932597,
  3157.6957665040204,
  3780.968387163944,
  2041.1032510923392,
  3717.5307063709497,
  3303.4957468508774,
  3252.980920717091,
  3666.4421809766823,
  4233.815545277124,
  2873.426575531749,
  2630.599858020261,
  3758.268358022575,
  3616.071559070958,
  2163.607903482686,
  2023.1014436354571,
  1547.5730666712568,
  3350.723819459836,
  4486.717283451711,
  3998.005747418414,
  2669.7578613416817,
  2230.649189464586,
  2574.0811684266864,
  3434.3666507639346,
  3532.723297680064,
  4198.701592159242,
  4028.975197502733,
  1745.5937227243485,
  1901.0397353275062,
  3983.9766091277324,
  3519.5977294619406,
  3464.7168587262327,
  4734.402766824991,
  3107.342568959926,
  4621.0967416481535,
  4299.975

In [18]:
dayPredict = np.array(dayPredict)
fitness = fitness_func(pop_size, day_predict, yPred, gene_per_pop, dayPredict)
fitness

[3.255167611919515e-05,
 1.991434838401368e-05,
 1.7551632954962496e-05,
 1.4251174694465162e-05,
 2.80837609868391e-05,
 1.8464640009445358e-05,
 1.672980394957857e-05,
 1.7778321452741746e-05,
 2.764227247426062e-05,
 1.7953663958099098e-05,
 2.0018395600566063e-05,
 2.5488918006560136e-05,
 1.5269775996651833e-05,
 2.2527497088933122e-05,
 1.8928966953267076e-05,
 3.515255973548221e-05,
 1.9022674896930556e-05,
 2.3108700722081426e-05,
 2.0931142703856974e-05,
 1.8348453885581675e-05,
 1.7308553984251255e-05,
 2.5464170580973025e-05,
 2.6608186480148333e-05,
 2.0019136891821378e-05,
 1.9930474169949704e-05,
 3.448469360431354e-05,
 3.527303275619469e-05,
 4.433284804434352e-05,
 2.0693394664973123e-05,
 1.5981663156573476e-05,
 1.84542870373674e-05,
 2.5678891377651243e-05,
 3.0504596531841866e-05,
 2.5365964823371874e-05,
 2.010703956740852e-05,
 2.1213527753887577e-05,
 1.6494160208663262e-05,
 1.797815205700025e-05,
 3.913267841456107e-05,
 3.671140971361663e-05,
 1.6942419443289

In [19]:
yProb = probability(fitness)
yProb

[2.8899051651379515,
 1.7679758807068366,
 1.558216374094525,
 1.2652049992145362,
 2.4932481398249027,
 1.6392722248868448,
 1.4852552190737334,
 1.5783415516187418,
 2.4540532323744295,
 1.593908283422197,
 1.7772130882605102,
 2.2628805819771456,
 1.3556354014348901,
 1.9999686024323737,
 1.6804947053621848,
 3.1208090045933465,
 1.688814002636338,
 2.0515672781897116,
 1.858245860846671,
 1.6289573373141888,
 1.5366360668187162,
 2.2606835303486466,
 2.362248114731971,
 1.7772788993548376,
 1.7694075118123376,
 3.061516519161332,
 3.1315044785696147,
 3.9358257952522298,
 1.837138828352983,
 1.4188360296581417,
 1.6383530984070571,
 2.2797462273695537,
 2.708167492052287,
 2.2519649216632245,
 1.7850828107503227,
 1.8833157224308477,
 1.4643350040436383,
 1.5960823122875527,
 3.474159949915311,
 3.259202142537766,
 1.5041310094082256,
 1.8331446585782463,
 1.8643851138468608,
 1.2966378340113347,
 2.013649342847099,
 1.3705100996043864,
 1.4146832275415477,
 1.8930875910064309,
 1.

In [20]:
yRanges = rangePc(yProb)
yRanges

array([[  0.        ,   2.88990517],
       [  2.88990517,   4.65788105],
       [  4.65788105,   6.21609742],
       [  6.21609742,   7.48130242],
       [  7.48130242,   9.97455056],
       [  9.97455056,  11.61382278],
       [ 11.61382278,  13.099078  ],
       [ 13.099078  ,  14.67741955],
       [ 14.67741955,  17.13147279],
       [ 17.13147279,  18.72538107],
       [ 18.72538107,  20.50259416],
       [ 20.50259416,  22.76547474],
       [ 22.76547474,  24.12111014],
       [ 24.12111014,  26.12107874],
       [ 26.12107874,  27.80157345],
       [ 27.80157345,  30.92238245],
       [ 30.92238245,  32.61119646],
       [ 32.61119646,  34.66276374],
       [ 34.66276374,  36.5210096 ],
       [ 36.5210096 ,  38.14996693],
       [ 38.14996693,  39.686603  ],
       [ 39.686603  ,  41.94728653],
       [ 41.94728653,  44.30953465],
       [ 44.30953465,  46.08681354],
       [ 46.08681354,  47.85622106],
       [ 47.85622106,  50.91773758],
       [ 50.91773758,  54.04924205],
 

In [21]:
roundWheel = round_wheel(pc, fitness, yRanges)
roundWheel

[34,
 0,
 49,
 3,
 42,
 41,
 30,
 41,
 31,
 41,
 27,
 18,
 47,
 27,
 12,
 49,
 9,
 31,
 41,
 24,
 26,
 10,
 11,
 5,
 13,
 42,
 16,
 27,
 17,
 26,
 11,
 36,
 38,
 18,
 20]

In [22]:
offSpring = cross_over(pc, roundWheel, generasiAwal)
offSpring

[[0.4489632744206763, 0.5155644816563111, 0.47976405251733145],
 [0.6343982577604932, 0.36885664002545415, 0.15119696972404598],
 [0.4078874808845149, 0.7272068383521575, 0.6678494090613103],
 [0.8029558509427187, 0.7580862070003309, 0.7199882891399736],
 [0.5890128437672599, 0.3590749144893278, 0.6131348059642748],
 [0.6029474206213801, 0.3627601708027094, 0.6167132911583082],
 [0.824178431778347, 0.2807099082512456, 0.7018564918365363],
 [0.6029474206213801, 0.3291751681292455, 0.6167132911583082],
 [0.6588305999981375, 0.39039268622457063, 0.22113630345365376],
 [0.6029474206213801, 0.37618207297495626, 0.6167132911583082],
 [0.14102507009072449, 0.4423744630069724, 0.23140292537730023],
 [0.6549506200984878, 0.5381548926309441, 0.3082459614736253],
 [0.5184689484187844, 0.3125059999401243, 0.725613522926825],
 [0.14102507009072449, 0.3456677947109068, 0.23140292537730023],
 [0.6184182249591573, 0.8000072612877849, 0.6926921284057463],
 [0.4078874808845149, 0.7451730044753522, 0.667

In [23]:
yPredOffspring = y_pred(pop_size, offSpring, dayPredict, gene_per_pop)
len(yPredOffspring)

48

In [24]:
fitnessOffspring = fitness_func(len(yPredOffspring[0]), day_predict, yPredOffspring, gene_per_pop, dayPredict)
fitnessOffspring

[2.516156389156663e-05,
 3.180659306728081e-05,
 2.0143552270311236e-05,
 1.5933347151461696e-05,
 2.3097077464998154e-05,
 2.279098024340569e-05,
 1.991310953631494e-05,
 2.326088950208353e-05,
 2.882018079296583e-05,
 2.2608453269578644e-05,
 4.501099685861875e-05,
 2.4384353252121543e-05,
 2.3046610912128704e-05,
 5.085345583493203e-05,
 1.7230645471138638e-05,
 1.9952972430913867e-05,
 1.919543316182105e-05,
 2.9785371624326686e-05,
 2.229079893764233e-05,
 2.2428021239135782e-05,
 3.482962908907499e-05,
 2.3671246759527505e-05,
 2.474810418287629e-05,
 2.287383672523165e-05,
 2.487307442556661e-05,
 2.3183618088754862e-05,
 2.232804740660738e-05,
 4.3261188072085525e-05,
 2.4443429010958133e-05,
 4.099050360356498e-05,
 2.485283183184667e-05,
 2.000246169301832e-05,
 4.012721113745637e-05,
 2.4411458402393935e-05,
 1.8606014888160377e-05]

In [25]:
evalIndividu = evalGen(generasiAwal, gene_per_pop, fitness, offSpring, fitnessOffspring)
evalIndividu

[[0.14102507009072449,
  0.3456677947109068,
  0.23140292537730023,
  5.085345583493203e-05],
 [0.14102507009072449,
  0.4423744630069724,
  0.23140292537730023,
  4.501099685861875e-05],
 [0.14102507009072449,
  0.37053914078899364,
  0.23140292537730023,
  4.433284804434352e-05],
 [0.14102507009072449,
  0.47642133407677334,
  0.23140292537730023,
  4.3261188072085525e-05],
 [0.19478472791985527,
  0.30509675449666307,
  0.3812930919459241,
  4.099050360356498e-05],
 [0.27149016075760324,
  0.43834874488304326,
  0.20528620663011996,
  4.012721113745637e-05],
 [0.27149016075760324,
  0.36478811489766627,
  0.20528620663011996,
  3.913267841456107e-05],
 [0.19073674145690314,
  0.3937760385703085,
  0.3111757308369857,
  3.671140971361663e-05],
 [0.19478472791985527,
  0.35418323663602125,
  0.3812930919459241,
  3.527303275619469e-05],
 [0.3167773262007647,
  0.2706042988391376,
  0.3450836894689807,
  3.515255973548221e-05],
 [0.19478472791985527,
  0.46859782555310836,
  0.38129309

In [26]:
evalElitism = eval_elitism(evalIndividu, pop_size)
evalElitism

[[0.14102507009072449,
  0.3456677947109068,
  0.23140292537730023,
  5.085345583493203e-05],
 [0.14102507009072449,
  0.4423744630069724,
  0.23140292537730023,
  4.501099685861875e-05],
 [0.14102507009072449,
  0.37053914078899364,
  0.23140292537730023,
  4.433284804434352e-05],
 [0.14102507009072449,
  0.47642133407677334,
  0.23140292537730023,
  4.3261188072085525e-05],
 [0.19478472791985527,
  0.30509675449666307,
  0.3812930919459241,
  4.099050360356498e-05],
 [0.27149016075760324,
  0.43834874488304326,
  0.20528620663011996,
  4.012721113745637e-05],
 [0.27149016075760324,
  0.36478811489766627,
  0.20528620663011996,
  3.913267841456107e-05],
 [0.19073674145690314,
  0.3937760385703085,
  0.3111757308369857,
  3.671140971361663e-05],
 [0.19478472791985527,
  0.35418323663602125,
  0.3812930919459241,
  3.527303275619469e-05],
 [0.3167773262007647,
  0.2706042988391376,
  0.3450836894689807,
  3.515255973548221e-05],
 [0.19478472791985527,
  0.46859782555310836,
  0.38129309

In [27]:
evalElitism[0][0]
d51 = round((evalElitism[0][0] * data[49]) + (evalElitism[0][1] * data[48]) + evalElitism[0][2])
d52 = round((evalElitism[1][0] * d51) + (evalElitism[1][1] * data[49]) + evalElitism[1][2])
print(f'prediksi covid pada hari ke-51 adalah sebanyak {d51}')
print(f'\nprediksi covid pada hari ke-52 adalah sebanyak {d52}')

prediksi covid pada hari ke-51 adalah sebanyak 14135

prediksi covid pada hari ke-52 adalah sebanyak 13644
